# Importing Packages

In [3]:
! pip install geopandas

     |████████████████████████████████| 1.0 MB 5.0 MB/s 
     |████████████████████████████████| 15.4 MB 77 kB/s 
     |████████████████████████████████| 6.3 MB 49.5 MB/s 


In [45]:
! pip install pygeocodio

  Created wheel for geocodio: filename=geocodio-1.0.1-py3-none-any.whl size=2559 sha256=98101546615e00e7be6330ccd7c8162e5e67c37ad74bfa931f98878456f17ffa
  Stored in directory: /root/.cache/pip/wheels/6f/62/a4/3c0be7b21d037aca6c570966aa952953c134fa134b104a2c83
Successfully built geocodio


In [70]:
import pandas as pd
from datetime import datetime
import numpy as np
import geopandas as gpd
from geocodio import GeocodioClient

# settings to display all columns
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 20)


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
data = pd.read_csv('/content/drive/MyDrive/Datasets/NYC Property Sales/df_nyc_housing.csv', index_col=0)
data.head()

,boro,NTA,bldg_cl_cat,tax_cl_pres,block,lot,bldg_cl_pres,address,apt_no,zip,res_un,com_un,total_un,land_sf,gross_sf,yr_blt,tax_cl_TOS,bldg_cl_TOS,sale_price,sale_date,sale_age,bc_comp
0,1,Alphabet City,03 Three Family Dwellings,1,376,24,C0,264 East 7th Street,,10009,3,0,3,2059,3696,1900-01-01,1.0,C0,7738700,2016-12-22,116,1
1,1,Alphabet City,09 Coops - Walkup Apartments,2,373,40,C6,"327 East 3rd Street, 3e",,10009,0,0,0,0,0,1920-01-01,2.0,C6,390000,2016-06-16,96,1
2,1,Alphabet City,09 Coops - Walkup Apartments,2,373,40,C6,"327 East 3 Street, 1c",,10009,0,0,0,0,0,1920-01-01,2.0,C6,1,2016-09-06,96,1
3,1,Alphabet City,09 Coops - Walkup Apartments,2,373,46,C6,"317 East 3rd Street, 9",,10009,0,0,0,0,0,1925-01-01,2.0,C6,635000,2016-02-18,91,1
4,1,Alphabet City,09 Coops - Walkup Apartments,2,373,49,C6,"311 East 3rd Street, 24",,10009,0,0,0,0,0,1920-01-01,2.0,C6,465000,2016-01-11,96,1


# Neighborhood (NTA) Names

In [71]:
data.NTA.value_counts()

Flushing-North             11242
Upper East Side (59-79)     8043
Upper East Side (79-96)     7019
Upper West Side (59-79)     6729
Bedford Stuyvesant          5378
                           ...  
Nan                            5
Bloomfield                     4
Bronx Park                     3
Fresh Kills                    3
Rossville-Port Mobil           2
Name: NTA, Length: 254, dtype: int64

## NTA Shapefile

In [7]:
nta = gpd.read_file("/content/drive/MyDrive/Datasets/NYC Property Sales/NYC_NTA_2020/nynta2020.shp")
nta.head()

,BoroCode,BoroName,CountyFIPS,NTA2020,NTAName,NTAAbbrev,NTAType,CDTA2020,CDTAName,Shape_Leng,Shape_Area,geometry
0,3,Brooklyn,047,BK0101,Greenpoint,Grnpt,0,BK01,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),28914.355027,3.532200e+07,"POLYGON ((1003059.997 204572.025, 1002950.775 ..."
1,3,Brooklyn,047,BK0102,Williamsburg,Wllmsbrg,0,BK01,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),28155.614845,2.886227e+07,"POLYGON ((995851.916 203199.332, 995969.192 20..."
2,3,Brooklyn,047,BK0103,South Williamsburg,SWllmsbrg,0,BK01,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),18252.346383,1.520934e+07,"POLYGON ((998047.210 196303.325, 998157.901 19..."
3,3,Brooklyn,047,BK0104,East Williamsburg,EWllmsbrg,0,BK01,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),43171.303949,5.226625e+07,"POLYGON ((1005302.497 199455.730, 1005307.792 ..."
4,3,Brooklyn,047,BK0201,Brooklyn Heights,BkHts,0,BK02,BK02 Downtown Brooklyn-Fort Greene (CD 2 Appro...,14337.911437,9.985391e+06,"POLYGON ((986367.735 190549.239, 985813.835 19..."


In [9]:
nta_key = nta[['NTA2020', 'NTAName']]
nta_key.head()

,NTA2020,NTAName
0,BK0101,Greenpoint
1,BK0102,Williamsburg
2,BK0103,South Williamsburg
3,BK0104,East Williamsburg
4,BK0201,Brooklyn Heights


In [11]:
queens = nta_key[nta_key.NTA2020.str.startswith('QN')].index
QN = nta_key.iloc[queens]
QN.head()

,NTA2020,NTAName
156,QN0101,Astoria (North)-Ditmars-Steinway
157,QN0102,Old Astoria-Hallets Point
158,QN0103,Astoria (Central)
159,QN0104,Astoria (East)-Woodside (North)
160,QN0105,Queensbridge-Ravenswood-Dutch Kills


In [72]:
nta_dict = dict(zip(nta_key.NTAName, nta_key.NTA2020))
nta_dict

{'Allerton': 'BX1104',
 'Alley Pond Park': 'QN1191',
 "Annadale-Huguenot-Prince's Bay-Woodrow": 'SI0304',
 'Arden Heights-Rossville': 'SI0303',
 'Astoria (Central)': 'QN0103',
 'Astoria (East)-Woodside (North)': 'QN0104',
 'Astoria (North)-Ditmars-Steinway': 'QN0101',
 'Astoria Park': 'QN0191',
 'Auburndale': 'QN1101',
 'Baisley Park': 'QN1203',
 'Barren Island-Floyd Bennett Field': 'BK5691',
 'Bath Beach': 'BK1102',
 'Bay Ridge': 'BK1001',
 'Bay Terrace-Clearview': 'QN0703',
 'Bayside': 'QN1102',
 'Bedford Park': 'BX0702',
 'Bedford-Stuyvesant (East)': 'BK0302',
 'Bedford-Stuyvesant (West)': 'BK0301',
 'Bellerose': 'QN1302',
 'Belmont': 'BX0603',
 'Bensonhurst': 'BK1101',
 'Borough Park': 'BK1202',
 'Breezy Point-Belle Harbor-Rockaway Park-Broad Channel': 'QN1403',
 'Brighton Beach': 'BK1303',
 'Bronx Park': 'BX2791',
 'Brooklyn Heights': 'BK0201',
 'Brooklyn Navy Yard': 'BK0261',
 'Brownsville': 'BK1602',
 'Bushwick (East)': 'BK0402',
 'Bushwick (West)': 'BK0401',
 'Calvary & Mount Z

In [73]:
nta_list = set(nta_dict.keys())
nta_list

{'Allerton',
 'Alley Pond Park',
 "Annadale-Huguenot-Prince's Bay-Woodrow",
 'Arden Heights-Rossville',
 'Astoria (Central)',
 'Astoria (East)-Woodside (North)',
 'Astoria (North)-Ditmars-Steinway',
 'Astoria Park',
 'Auburndale',
 'Baisley Park',
 'Barren Island-Floyd Bennett Field',
 'Bath Beach',
 'Bay Ridge',
 'Bay Terrace-Clearview',
 'Bayside',
 'Bedford Park',
 'Bedford-Stuyvesant (East)',
 'Bedford-Stuyvesant (West)',
 'Bellerose',
 'Belmont',
 'Bensonhurst',
 'Borough Park',
 'Breezy Point-Belle Harbor-Rockaway Park-Broad Channel',
 'Brighton Beach',
 'Bronx Park',
 'Brooklyn Heights',
 'Brooklyn Navy Yard',
 'Brownsville',
 'Bushwick (East)',
 'Bushwick (West)',
 'Calvary & Mount Zion Cemeteries',
 'Calvert Vaux Park',
 'Cambria Heights',
 'Canarsie',
 'Canarsie Park & Pier',
 'Carroll Gardens-Cobble Hill-Gowanus-Red Hook',
 'Castle Hill-Unionport',
 'Central Park',
 'Chelsea-Hudson Yards',
 'Chinatown-Two Bridges',
 'Claremont Park',
 'Claremont Village-Claremont (East)',
 '

In [75]:
data.NTA = data.NTA.apply(lambda x: x.strip())

In [74]:
nghbrhd_list = set(data.NTA.values)
nghbrhd_list

{'Airport La Guardia',
 'Alphabet City',
 'Annadale',
 'Arden Heights',
 'Arrochar',
 'Arrochar-Shore Acres',
 'Arverne',
 'Astoria',
 'Bath Beach',
 'Bathgate',
 'Bay Ridge',
 'Baychester',
 'Bayside',
 'Bedford Park/Norwood',
 'Bedford Stuyvesant',
 'Beechhurst',
 'Belle Harbor',
 'Bellerose',
 'Belmont',
 'Bensonhurst',
 'Bergen Beach',
 'Bloomfield',
 'Boerum Hill',
 'Borough Park',
 'Breezy Point',
 'Briarwood',
 'Brighton Beach',
 'Broad Channel',
 'Bronx Park',
 'Bronxdale',
 'Brooklyn Heights',
 'Brownsville',
 'Bulls Head',
 'Bush Terminal',
 'Bushwick',
 'Cambria Heights',
 'Canarsie',
 'Carroll Gardens',
 'Castle Hill/Unionport',
 'Castleton Corners',
 'Chelsea',
 'Chinatown',
 'City Island',
 'City Island-Pelham Strip',
 'Civic Center',
 'Clinton',
 'Clinton Hill',
 'Clove Lakes',
 'Co-Op City',
 'Cobble Hill',
 'Cobble Hill-West',
 'College Point',
 'Concord',
 'Concord-Fox Hills',
 'Coney Island',
 'Corona',
 'Country Club',
 'Crotona Park',
 'Crown Heights',
 'Cypress Hi

In [17]:
diff_1 = nta_list - nghbrhd_list
diff_1

{'Allerton',
 'Alley Pond Park',
 "Annadale-Huguenot-Prince's Bay-Woodrow",
 'Arden Heights-Rossville',
 'Astoria (Central)',
 'Astoria (East)-Woodside (North)',
 'Astoria (North)-Ditmars-Steinway',
 'Astoria Park',
 'Auburndale',
 'Baisley Park',
 'Barren Island-Floyd Bennett Field',
 'Bay Terrace-Clearview',
 'Bedford Park',
 'Bedford-Stuyvesant (East)',
 'Bedford-Stuyvesant (West)',
 'Breezy Point-Belle Harbor-Rockaway Park-Broad Channel',
 'Brooklyn Navy Yard',
 'Bushwick (East)',
 'Bushwick (West)',
 'Calvary & Mount Zion Cemeteries',
 'Calvert Vaux Park',
 'Canarsie Park & Pier',
 'Carroll Gardens-Cobble Hill-Gowanus-Red Hook',
 'Castle Hill-Unionport',
 'Central Park',
 'Chelsea-Hudson Yards',
 'Chinatown-Two Bridges',
 'Claremont Park',
 'Claremont Village-Claremont (East)',
 'Co-op City',
 'Concourse-Concourse Village',
 'Coney Island-Sea Gate',
 'Crotona Park East',
 'Crown Heights (North)',
 'Crown Heights (South)',
 'Cunningham Park',
 'Douglaston-Little Neck',
 'Downtown B

In [18]:
diff_2 = nghbrhd_list - nta_list
diff_2

{'Airport La Guardia',
 'Alphabet City',
 'Annadale',
 'Arden Heights',
 'Arrochar',
 'Arrochar-Shore Acres',
 'Arverne',
 'Astoria',
 'Bathgate',
 'Baychester',
 'Bedford Park/Norwood',
 'Bedford Stuyvesant',
 'Beechhurst',
 'Belle Harbor',
 'Bergen Beach',
 'Bloomfield',
 'Boerum Hill',
 'Breezy Point',
 'Briarwood',
 'Broad Channel',
 'Bronxdale',
 'Bulls Head',
 'Bush Terminal',
 'Bushwick',
 'Carroll Gardens',
 'Castle Hill/Unionport',
 'Castleton Corners',
 'Chelsea',
 'Chinatown',
 'City Island',
 'City Island-Pelham Strip',
 'Civic Center',
 'Clinton',
 'Clove Lakes',
 'Co-Op City',
 'Cobble Hill',
 'Cobble Hill-West',
 'Concord',
 'Concord-Fox Hills',
 'Coney Island',
 'Country Club',
 'Crown Heights',
 'Dongan Hills',
 'Dongan Hills-Colony',
 'Dongan Hills-Old Town',
 'Douglaston',
 'Downtown-Fulton Ferry',
 'Downtown-Fulton Mall',
 'Downtown-Metrotech',
 'East New York',
 'East Tremont',
 'Eltingville',
 'Emerson Hill',
 'Far Rockaway',
 'Fashion',
 'Fieldston',
 'Financial'

In [19]:
len(diff_2)

193

# Geomapping Addresses

In [20]:
data.boro.value_counts()

4    115472
3     94490
1     74637
5     38985
2     30933
0         5
Name: boro, dtype: int64

## Manhattan

In [22]:
data_man = data[data.boro == 1].reset_index(drop=True)
data_man.head()

,boro,NTA,bldg_cl_cat,tax_cl_pres,block,lot,bldg_cl_pres,address,apt_no,zip,res_un,com_un,total_un,land_sf,gross_sf,yr_blt,tax_cl_TOS,bldg_cl_TOS,sale_price,sale_date,sale_age,bc_comp
0,1,Alphabet City,03 Three Family Dwellings,1,376,24,C0,264 East 7th Street,,10009,3,0,3,2059,3696,1900-01-01,1.0,C0,7738700,2016-12-22,116,1
1,1,Alphabet City,09 Coops - Walkup Apartments,2,373,40,C6,"327 East 3rd Street, 3e",,10009,0,0,0,0,0,1920-01-01,2.0,C6,390000,2016-06-16,96,1
2,1,Alphabet City,09 Coops - Walkup Apartments,2,373,40,C6,"327 East 3 Street, 1c",,10009,0,0,0,0,0,1920-01-01,2.0,C6,1,2016-09-06,96,1
3,1,Alphabet City,09 Coops - Walkup Apartments,2,373,46,C6,"317 East 3rd Street, 9",,10009,0,0,0,0,0,1925-01-01,2.0,C6,635000,2016-02-18,91,1
4,1,Alphabet City,09 Coops - Walkup Apartments,2,373,49,C6,"311 East 3rd Street, 24",,10009,0,0,0,0,0,1920-01-01,2.0,C6,465000,2016-01-11,96,1


In [23]:
addy = str(data_man.address[0]) + ', New York, NY ' + str(data_man.zip[0])
addy

'264 East 7th Street, New York, NY 10009'

In [24]:
data_man['full_addr'] = data_man.address.astype(str) + ', New York, NY ' + data_man.zip.astype(str)

In [25]:
data_man.head()

,boro,NTA,bldg_cl_cat,tax_cl_pres,block,lot,bldg_cl_pres,address,apt_no,zip,res_un,com_un,total_un,land_sf,gross_sf,yr_blt,tax_cl_TOS,bldg_cl_TOS,sale_price,sale_date,sale_age,bc_comp,full_addr
0,1,Alphabet City,03 Three Family Dwellings,1,376,24,C0,264 East 7th Street,,10009,3,0,3,2059,3696,1900-01-01,1.0,C0,7738700,2016-12-22,116,1,"264 East 7th Street, New York, NY 10009"
1,1,Alphabet City,09 Coops - Walkup Apartments,2,373,40,C6,"327 East 3rd Street, 3e",,10009,0,0,0,0,0,1920-01-01,2.0,C6,390000,2016-06-16,96,1,"327 East 3rd Street, 3e, New York, NY 10009"
2,1,Alphabet City,09 Coops - Walkup Apartments,2,373,40,C6,"327 East 3 Street, 1c",,10009,0,0,0,0,0,1920-01-01,2.0,C6,1,2016-09-06,96,1,"327 East 3 Street, 1c, New York, NY 10009"
3,1,Alphabet City,09 Coops - Walkup Apartments,2,373,46,C6,"317 East 3rd Street, 9",,10009,0,0,0,0,0,1925-01-01,2.0,C6,635000,2016-02-18,91,1,"317 East 3rd Street, 9, New York, NY 10009"
4,1,Alphabet City,09 Coops - Walkup Apartments,2,373,49,C6,"311 East 3rd Street, 24",,10009,0,0,0,0,0,1920-01-01,2.0,C6,465000,2016-01-11,96,1,"311 East 3rd Street, 24, New York, NY 10009"


In [76]:
data_man.full_addr.isna().sum()

0

## Bronx

In [26]:
data_bx = data[data.boro == 2].reset_index(drop=True)
data_bx.head()

,boro,NTA,bldg_cl_cat,tax_cl_pres,block,lot,bldg_cl_pres,address,apt_no,zip,res_un,com_un,total_un,land_sf,gross_sf,yr_blt,tax_cl_TOS,bldg_cl_TOS,sale_price,sale_date,sale_age,bc_comp
0,2,Bathgate,01 One Family Dwellings,1,3037,101,S0,443 East 180 Street,,10457,1,2,3,1293,2310,1952-01-01,1.0,S0,0,2016-08-06,64,1
1,2,Bathgate,01 One Family Dwellings,1,3043,55,A1,1948 Bathgate Avenue,,10457,1,0,1,2356,2047,1901-01-01,1.0,A1,220000,2016-06-22,115,1
2,2,Bathgate,01 One Family Dwellings,1,3046,52,A1,2047 Bathgate Avenue,,10457,1,0,1,2329,1431,1901-01-01,1.0,A1,273796,2016-09-21,115,1
3,2,Bathgate,01 One Family Dwellings,1,3046,52,A1,2047 Bathgate Avenue,,10457,1,0,1,2329,1431,1901-01-01,1.0,A1,231244,2016-09-21,115,1
4,2,Bathgate,01 One Family Dwellings,1,3048,27,A1,538 East 182 Street,,10457,1,0,1,1209,1048,1901-01-01,1.0,A1,0,2016-09-20,115,1


In [27]:
data_bx['full_addr'] = data_bx.address.astype(str) + ', Bronx, NY ' + data_bx.zip.astype(str)
data_bx.head()

,boro,NTA,bldg_cl_cat,tax_cl_pres,block,lot,bldg_cl_pres,address,apt_no,zip,res_un,com_un,total_un,land_sf,gross_sf,yr_blt,tax_cl_TOS,bldg_cl_TOS,sale_price,sale_date,sale_age,bc_comp,full_addr
0,2,Bathgate,01 One Family Dwellings,1,3037,101,S0,443 East 180 Street,,10457,1,2,3,1293,2310,1952-01-01,1.0,S0,0,2016-08-06,64,1,"443 East 180 Street, Bronx, NY 10457"
1,2,Bathgate,01 One Family Dwellings,1,3043,55,A1,1948 Bathgate Avenue,,10457,1,0,1,2356,2047,1901-01-01,1.0,A1,220000,2016-06-22,115,1,"1948 Bathgate Avenue, Bronx, NY 10457"
2,2,Bathgate,01 One Family Dwellings,1,3046,52,A1,2047 Bathgate Avenue,,10457,1,0,1,2329,1431,1901-01-01,1.0,A1,273796,2016-09-21,115,1,"2047 Bathgate Avenue, Bronx, NY 10457"
3,2,Bathgate,01 One Family Dwellings,1,3046,52,A1,2047 Bathgate Avenue,,10457,1,0,1,2329,1431,1901-01-01,1.0,A1,231244,2016-09-21,115,1,"2047 Bathgate Avenue, Bronx, NY 10457"
4,2,Bathgate,01 One Family Dwellings,1,3048,27,A1,538 East 182 Street,,10457,1,0,1,1209,1048,1901-01-01,1.0,A1,0,2016-09-20,115,1,"538 East 182 Street, Bronx, NY 10457"


## Brooklyn

In [28]:
data_bk = data[data.boro == 3].reset_index(drop=True)
data_bk.head()

,boro,NTA,bldg_cl_cat,tax_cl_pres,block,lot,bldg_cl_pres,address,apt_no,zip,res_un,com_un,total_un,land_sf,gross_sf,yr_blt,tax_cl_TOS,bldg_cl_TOS,sale_price,sale_date,sale_age,bc_comp
0,3,Bath Beach,01 One Family Dwellings,1,6360,13,A5,8665 15th Avenue,,11228,1,0,1,1547,2224,1930-01-01,1.0,A5,0,2016-05-25,86,1
1,3,Bath Beach,01 One Family Dwellings,1,6361,11,A5,71 Bay 10th Street,,11228,1,0,1,2900,1660,1930-01-01,1.0,A5,829000,2016-04-05,86,1
2,3,Bath Beach,01 One Family Dwellings,1,6364,2,A5,1649 Benson Avenue,,11214,1,0,1,1638,972,1930-01-01,1.0,A5,0,2016-10-06,86,1
3,3,Bath Beach,01 One Family Dwellings,1,6364,72,A5,68 Bay 14th Street,,11214,1,0,1,1950,972,1950-01-01,1.0,A5,790000,2016-06-21,66,1
4,3,Bath Beach,01 One Family Dwellings,1,6371,79,A5,8668 19th Avenue,,11214,1,0,1,2223,2520,1930-01-01,1.0,A5,788000,2016-03-31,86,1


In [29]:
data_bk['full_addr'] = data_bk.address.astype(str) + ', Brooklyn, NY ' + data_bk.zip.astype(str)
data_bk.head()

,boro,NTA,bldg_cl_cat,tax_cl_pres,block,lot,bldg_cl_pres,address,apt_no,zip,res_un,com_un,total_un,land_sf,gross_sf,yr_blt,tax_cl_TOS,bldg_cl_TOS,sale_price,sale_date,sale_age,bc_comp,full_addr
0,3,Bath Beach,01 One Family Dwellings,1,6360,13,A5,8665 15th Avenue,,11228,1,0,1,1547,2224,1930-01-01,1.0,A5,0,2016-05-25,86,1,"8665 15th Avenue, Brooklyn, NY 11228"
1,3,Bath Beach,01 One Family Dwellings,1,6361,11,A5,71 Bay 10th Street,,11228,1,0,1,2900,1660,1930-01-01,1.0,A5,829000,2016-04-05,86,1,"71 Bay 10th Street, Brooklyn, NY 11228"
2,3,Bath Beach,01 One Family Dwellings,1,6364,2,A5,1649 Benson Avenue,,11214,1,0,1,1638,972,1930-01-01,1.0,A5,0,2016-10-06,86,1,"1649 Benson Avenue, Brooklyn, NY 11214"
3,3,Bath Beach,01 One Family Dwellings,1,6364,72,A5,68 Bay 14th Street,,11214,1,0,1,1950,972,1950-01-01,1.0,A5,790000,2016-06-21,66,1,"68 Bay 14th Street, Brooklyn, NY 11214"
4,3,Bath Beach,01 One Family Dwellings,1,6371,79,A5,8668 19th Avenue,,11214,1,0,1,2223,2520,1930-01-01,1.0,A5,788000,2016-03-31,86,1,"8668 19th Avenue, Brooklyn, NY 11214"


## Queens

In [30]:
data_qns = data[data.boro == 4].reset_index(drop=True)
data_qns.head()

,boro,NTA,bldg_cl_cat,tax_cl_pres,block,lot,bldg_cl_pres,address,apt_no,zip,res_un,com_un,total_un,land_sf,gross_sf,yr_blt,tax_cl_TOS,bldg_cl_TOS,sale_price,sale_date,sale_age,bc_comp
0,4,Airport La Guardia,01 One Family Dwellings,1,976,61,A5,21-21 80th Street,,11370,1,0,1,1800,1836,1950-01-01,1.0,A5,660000,2016-07-26,66,1
1,4,Airport La Guardia,01 One Family Dwellings,1,976,63,A5,21-17 80th Street,,11370,1,0,1,1800,1224,1950-01-01,1.0,A5,275500,2016-11-18,66,1
2,4,Airport La Guardia,02 Two Family Dwellings,1,976,70,B1,21-03 80th Street,,11370,2,0,2,1800,1836,1950-01-01,1.0,B1,0,2016-06-13,66,1
3,4,Airport La Guardia,03 Three Family Dwellings,1,949,15,C0,19-08 81st Street,,11370,3,0,3,2112,4300,1985-01-01,1.0,C0,940000,2016-04-14,31,1
4,4,Airport La Guardia,03 Three Family Dwellings,1,949,56,C0,19-69 80th Street,,11370,3,0,3,2000,2835,1945-01-01,1.0,C0,0,2016-08-15,71,1


In [31]:
data_qns.NTA.value_counts()

Flushing-North              11242
Bayside                      5330
Forest Hills                 4879
Jackson Heights              4611
Flushing-South               3924
Astoria                      3905
Richmond Hill                3586
Elmhurst                     3363
St. Albans                   3145
Queens Village               3141
So. Jamaica-Baisley Park     3117
Rego Park                    3106
South Ozone Park             2711
Springfield Gardens          2706
South Jamaica                2593
Howard Beach                 2532
Corona                       2443
Ozone Park                   2400
Long Island City             2395
Middle Village               2258
Hollis                       2159
Rosedale                     2107
Whitestone                   2069
Oakland Gardens              1936
Woodhaven                    1909
Far Rockaway                 1886
Glendale                     1814
Laurelton                    1800
Glen Oaks                    1684
College Point 

In [34]:
data_qns['City'] = data_qns.NTA
data_qns.head()

,boro,NTA,bldg_cl_cat,tax_cl_pres,block,lot,bldg_cl_pres,address,apt_no,zip,res_un,com_un,total_un,land_sf,gross_sf,yr_blt,tax_cl_TOS,bldg_cl_TOS,sale_price,sale_date,sale_age,bc_comp,City
0,4,Airport La Guardia,01 One Family Dwellings,1,976,61,A5,21-21 80th Street,,11370,1,0,1,1800,1836,1950-01-01,1.0,A5,660000,2016-07-26,66,1,Airport La Guardia
1,4,Airport La Guardia,01 One Family Dwellings,1,976,63,A5,21-17 80th Street,,11370,1,0,1,1800,1224,1950-01-01,1.0,A5,275500,2016-11-18,66,1,Airport La Guardia
2,4,Airport La Guardia,02 Two Family Dwellings,1,976,70,B1,21-03 80th Street,,11370,2,0,2,1800,1836,1950-01-01,1.0,B1,0,2016-06-13,66,1,Airport La Guardia
3,4,Airport La Guardia,03 Three Family Dwellings,1,949,15,C0,19-08 81st Street,,11370,3,0,3,2112,4300,1985-01-01,1.0,C0,940000,2016-04-14,31,1,Airport La Guardia
4,4,Airport La Guardia,03 Three Family Dwellings,1,949,56,C0,19-69 80th Street,,11370,3,0,3,2000,2835,1945-01-01,1.0,C0,0,2016-08-15,71,1,Airport La Guardia


In [35]:
data_qns.City = data_qns.City.replace({'Flushing-North': 'Flushing', 
                                        'Flushing-South': 'Flushing', 
                                        'So. Jamaica-Baisley Park': 'Jamaica', 
                                        'Beechhurst': 'Whitestone', 
                                        'Briarwood': 'Jamaica', 
                                        'Jamaica Estates' : 'Jamaica', 
                                        'Hillcrest': 'Queens', 
                                        'Belle Harbor': 'Rockaway Park', 
                                        'Jamaica Hills': 'Jamaica', 
                                        'Holliswood': 'Jamaica',
                                        'Hammels': 'Rockaway Beach',
                                        'Hollis Hills': 'Queens Village',
                                        'Broad Channel': 'Rockaway Beach',
                                        'Neponsit': 'Rockaway Beach',
                                        'Jamaica Bay': 'Rockaway Beach',
                                        'Flushing Meadow Park': 'Queens',
                                        'Airport La Guardia': 'Queens',
                                        'Queens-Unknown': 'Queens'})
data_qns.head()


,boro,NTA,bldg_cl_cat,tax_cl_pres,block,lot,bldg_cl_pres,address,apt_no,zip,res_un,com_un,total_un,land_sf,gross_sf,yr_blt,tax_cl_TOS,bldg_cl_TOS,sale_price,sale_date,sale_age,bc_comp,City
0,4,Airport La Guardia,01 One Family Dwellings,1,976,61,A5,21-21 80th Street,,11370,1,0,1,1800,1836,1950-01-01,1.0,A5,660000,2016-07-26,66,1,Queens
1,4,Airport La Guardia,01 One Family Dwellings,1,976,63,A5,21-17 80th Street,,11370,1,0,1,1800,1224,1950-01-01,1.0,A5,275500,2016-11-18,66,1,Queens
2,4,Airport La Guardia,02 Two Family Dwellings,1,976,70,B1,21-03 80th Street,,11370,2,0,2,1800,1836,1950-01-01,1.0,B1,0,2016-06-13,66,1,Queens
3,4,Airport La Guardia,03 Three Family Dwellings,1,949,15,C0,19-08 81st Street,,11370,3,0,3,2112,4300,1985-01-01,1.0,C0,940000,2016-04-14,31,1,Queens
4,4,Airport La Guardia,03 Three Family Dwellings,1,949,56,C0,19-69 80th Street,,11370,3,0,3,2000,2835,1945-01-01,1.0,C0,0,2016-08-15,71,1,Queens


In [36]:
data_qns['full_addr'] = data_qns.address.astype(str) + ', ' + data_qns.City.astype(str) + ', NY ' + data_qns.zip.astype(str)
data_qns.head()

,boro,NTA,bldg_cl_cat,tax_cl_pres,block,lot,bldg_cl_pres,address,apt_no,zip,res_un,com_un,total_un,land_sf,gross_sf,yr_blt,tax_cl_TOS,bldg_cl_TOS,sale_price,sale_date,sale_age,bc_comp,City,full_addr
0,4,Airport La Guardia,01 One Family Dwellings,1,976,61,A5,21-21 80th Street,,11370,1,0,1,1800,1836,1950-01-01,1.0,A5,660000,2016-07-26,66,1,Queens,"21-21 80th Street, Queens, NY 11370"
1,4,Airport La Guardia,01 One Family Dwellings,1,976,63,A5,21-17 80th Street,,11370,1,0,1,1800,1224,1950-01-01,1.0,A5,275500,2016-11-18,66,1,Queens,"21-17 80th Street, Queens, NY 11370"
2,4,Airport La Guardia,02 Two Family Dwellings,1,976,70,B1,21-03 80th Street,,11370,2,0,2,1800,1836,1950-01-01,1.0,B1,0,2016-06-13,66,1,Queens,"21-03 80th Street, Queens, NY 11370"
3,4,Airport La Guardia,03 Three Family Dwellings,1,949,15,C0,19-08 81st Street,,11370,3,0,3,2112,4300,1985-01-01,1.0,C0,940000,2016-04-14,31,1,Queens,"19-08 81st Street, Queens, NY 11370"
4,4,Airport La Guardia,03 Three Family Dwellings,1,949,56,C0,19-69 80th Street,,11370,3,0,3,2000,2835,1945-01-01,1.0,C0,0,2016-08-15,71,1,Queens,"19-69 80th Street, Queens, NY 11370"


## Staten Island

In [37]:
data_si = data[data.boro == 5].reset_index(drop=True)
data_si.head()

,boro,NTA,bldg_cl_cat,tax_cl_pres,block,lot,bldg_cl_pres,address,apt_no,zip,res_un,com_un,total_un,land_sf,gross_sf,yr_blt,tax_cl_TOS,bldg_cl_TOS,sale_price,sale_date,sale_age,bc_comp
0,5,Annadale,01 One Family Dwellings,1,5391,9,A5,4726 Amboy Road,,10312,1,0,1,3067,1110,2002-01-01,1.0,A5,520000,2016-04-21,14,1
1,5,Annadale,01 One Family Dwellings,1,5391,30,A1,1306 Arden Avenue,,10312,1,0,1,7500,1890,1952-01-01,1.0,A1,0,2016-05-04,64,1
2,5,Annadale,01 One Family Dwellings,1,5395,15,A1,10 Edwin Street,,10312,1,0,1,7000,2200,1980-01-01,1.0,A1,10,2016-04-21,36,1
3,5,Annadale,01 One Family Dwellings,1,5397,60,A1,33 Eylandt Street,,10312,1,0,1,10120,3516,1984-01-01,1.0,A1,0,2016-08-10,32,1
4,5,Annadale,01 One Family Dwellings,1,5399,26,A1,235 Koch Boulevard,,10312,1,0,1,9555,1528,1960-01-01,1.0,A1,0,2016-02-11,56,1


In [38]:
data_si['full_addr'] = data_si.address.astype(str) + ', Staten Island, NY ' + data_si.zip.astype(str)
data_si.head()

,boro,NTA,bldg_cl_cat,tax_cl_pres,block,lot,bldg_cl_pres,address,apt_no,zip,res_un,com_un,total_un,land_sf,gross_sf,yr_blt,tax_cl_TOS,bldg_cl_TOS,sale_price,sale_date,sale_age,bc_comp,full_addr
0,5,Annadale,01 One Family Dwellings,1,5391,9,A5,4726 Amboy Road,,10312,1,0,1,3067,1110,2002-01-01,1.0,A5,520000,2016-04-21,14,1,"4726 Amboy Road, Staten Island, NY 10312"
1,5,Annadale,01 One Family Dwellings,1,5391,30,A1,1306 Arden Avenue,,10312,1,0,1,7500,1890,1952-01-01,1.0,A1,0,2016-05-04,64,1,"1306 Arden Avenue, Staten Island, NY 10312"
2,5,Annadale,01 One Family Dwellings,1,5395,15,A1,10 Edwin Street,,10312,1,0,1,7000,2200,1980-01-01,1.0,A1,10,2016-04-21,36,1,"10 Edwin Street, Staten Island, NY 10312"
3,5,Annadale,01 One Family Dwellings,1,5397,60,A1,33 Eylandt Street,,10312,1,0,1,10120,3516,1984-01-01,1.0,A1,0,2016-08-10,32,1,"33 Eylandt Street, Staten Island, NY 10312"
4,5,Annadale,01 One Family Dwellings,1,5399,26,A1,235 Koch Boulevard,,10312,1,0,1,9555,1528,1960-01-01,1.0,A1,0,2016-02-11,56,1,"235 Koch Boulevard, Staten Island, NY 10312"


In [41]:
data_zero = data[data.boro == 0].reset_index(drop=True)
data_zero['full_addr'] = np.nan

In [40]:
data_qns.drop(columns=['City'], inplace=True)

In [42]:
df = pd.concat([data_man, data_bx, data_qns, data_bk, data_bx, data_zero])
df.head()

,boro,NTA,bldg_cl_cat,tax_cl_pres,block,lot,bldg_cl_pres,address,apt_no,zip,res_un,com_un,total_un,land_sf,gross_sf,yr_blt,tax_cl_TOS,bldg_cl_TOS,sale_price,sale_date,sale_age,bc_comp,full_addr
0,1,Alphabet City,03 Three Family Dwellings,1,376,24,C0,264 East 7th Street,,10009,3,0,3,2059,3696,1900-01-01,1.0,C0,7738700,2016-12-22,116,1,"264 East 7th Street, New York, NY 10009"
1,1,Alphabet City,09 Coops - Walkup Apartments,2,373,40,C6,"327 East 3rd Street, 3e",,10009,0,0,0,0,0,1920-01-01,2.0,C6,390000,2016-06-16,96,1,"327 East 3rd Street, 3e, New York, NY 10009"
2,1,Alphabet City,09 Coops - Walkup Apartments,2,373,40,C6,"327 East 3 Street, 1c",,10009,0,0,0,0,0,1920-01-01,2.0,C6,1,2016-09-06,96,1,"327 East 3 Street, 1c, New York, NY 10009"
3,1,Alphabet City,09 Coops - Walkup Apartments,2,373,46,C6,"317 East 3rd Street, 9",,10009,0,0,0,0,0,1925-01-01,2.0,C6,635000,2016-02-18,91,1,"317 East 3rd Street, 9, New York, NY 10009"
4,1,Alphabet City,09 Coops - Walkup Apartments,2,373,49,C6,"311 East 3rd Street, 24",,10009,0,0,0,0,0,1920-01-01,2.0,C6,465000,2016-01-11,96,1,"311 East 3rd Street, 24, New York, NY 10009"


# Geocodio

In [91]:

API_KEY = '6fe1a9886948111a4aa1841fed9dfd48a568ea6'
API_KEY_2 = 'acb9b15c898cbcc68c7ce57d1719cc5be6aeca6'
client = GeocodioClient(API_KEY_2)

In [82]:
data_man.head()

,boro,NTA,bldg_cl_cat,tax_cl_pres,block,lot,bldg_cl_pres,address,apt_no,zip,res_un,com_un,total_un,land_sf,gross_sf,yr_blt,tax_cl_TOS,bldg_cl_TOS,sale_price,sale_date,sale_age,bc_comp,full_addr
0,1,Alphabet City,03 Three Family Dwellings,1,376,24,C0,264 East 7th Street,,10009,3,0,3,2059,3696,1900-01-01,1.0,C0,7738700,2016-12-22,116,1,"264 East 7th Street, New York, NY 10009"
1,1,Alphabet City,09 Coops - Walkup Apartments,2,373,40,C6,"327 East 3rd Street, 3e",,10009,0,0,0,0,0,1920-01-01,2.0,C6,390000,2016-06-16,96,1,"327 East 3rd Street, 3e, New York, NY 10009"
2,1,Alphabet City,09 Coops - Walkup Apartments,2,373,40,C6,"327 East 3 Street, 1c",,10009,0,0,0,0,0,1920-01-01,2.0,C6,1,2016-09-06,96,1,"327 East 3 Street, 1c, New York, NY 10009"
3,1,Alphabet City,09 Coops - Walkup Apartments,2,373,46,C6,"317 East 3rd Street, 9",,10009,0,0,0,0,0,1925-01-01,2.0,C6,635000,2016-02-18,91,1,"317 East 3rd Street, 9, New York, NY 10009"
4,1,Alphabet City,09 Coops - Walkup Apartments,2,373,49,C6,"311 East 3rd Street, 24",,10009,0,0,0,0,0,1920-01-01,2.0,C6,465000,2016-01-11,96,1,"311 East 3rd Street, 24, New York, NY 10009"


In [81]:
addy = data_man['full_addr'][0]
geocoded_location = client.geocode(addy)
geocoded_location.coords

GeocodioAuthError: ignored

In [53]:
geolist = list(df.full_addr.values)
len(geolist)

346470

In [54]:
geoman = list(data_man.full_addr.values)
len(geoman)

74637

In [66]:
geoman[0]

'264 East 7th Street, New York, NY 10009'

2489: 0 - 2488


In [92]:
man_coord = []

for i in range(2490, 5000):
    geocoded_location = client.geocode(geoman[i])
    man_coord.append(geocoded_location.coords)


GeocodioAuthError: ignored

In [85]:
len(man_coord_9000)

2489

In [86]:
   # Save a dictionary into a pickle file.
   import pickle
 
   pickle.dump(man_coord_9000, open("manh_0-2488.pickle", "wb"))

In [58]:
geoman_1 = geoman[0:9000]
geoman_2 = geoman[9000:18000]
geoman_3 = geoman[18000:27000]
geoman_4 = geoman[27000:36000]
geoman_5 = geoman[36000:45000]
geoman_6 = geoman[45000:54000]
geoman_7 = geoman[54000:63000]
geoman_8 = geoman[63000:72000]
geoman_9 = geoman[72000:]

In [93]:
data_man.head()

,boro,NTA,bldg_cl_cat,tax_cl_pres,block,lot,bldg_cl_pres,address,apt_no,zip,res_un,com_un,total_un,land_sf,gross_sf,yr_blt,tax_cl_TOS,bldg_cl_TOS,sale_price,sale_date,sale_age,bc_comp,full_addr
0,1,Alphabet City,03 Three Family Dwellings,1,376,24,C0,264 East 7th Street,,10009,3,0,3,2059,3696,1900-01-01,1.0,C0,7738700,2016-12-22,116,1,"264 East 7th Street, New York, NY 10009"
1,1,Alphabet City,09 Coops - Walkup Apartments,2,373,40,C6,"327 East 3rd Street, 3e",,10009,0,0,0,0,0,1920-01-01,2.0,C6,390000,2016-06-16,96,1,"327 East 3rd Street, 3e, New York, NY 10009"
2,1,Alphabet City,09 Coops - Walkup Apartments,2,373,40,C6,"327 East 3 Street, 1c",,10009,0,0,0,0,0,1920-01-01,2.0,C6,1,2016-09-06,96,1,"327 East 3 Street, 1c, New York, NY 10009"
3,1,Alphabet City,09 Coops - Walkup Apartments,2,373,46,C6,"317 East 3rd Street, 9",,10009,0,0,0,0,0,1925-01-01,2.0,C6,635000,2016-02-18,91,1,"317 East 3rd Street, 9, New York, NY 10009"
4,1,Alphabet City,09 Coops - Walkup Apartments,2,373,49,C6,"311 East 3rd Street, 24",,10009,0,0,0,0,0,1920-01-01,2.0,C6,465000,2016-01-11,96,1,"311 East 3rd Street, 24, New York, NY 10009"


In [94]:
man_upload = data_man[['block', 'lot', 'full_addr']]
man_upload

,block,lot,full_addr
0,376,24,"264 East 7th Street, New York, NY 10009"
1,373,40,"327 East 3rd Street, 3e, New York, NY 10009"
2,373,40,"327 East 3 Street, 1c, New York, NY 10009"
3,373,46,"317 East 3rd Street, 9, New York, NY 10009"
4,373,49,"311 East 3rd Street, 24, New York, NY 10009"
...,...,...,...
74632,2180,1245,"736 West 187 Street, 208, New York, NY 10033"
74633,2180,1248,"736 West 187th Street, 302, New York, NY 10033"
74634,2180,1283,"736 West 187 Street, Ph1, New York, NY 10033"
74635,2180,1429,"69 Bennett Avenue, 309, New York, NY 10033"


In [95]:
man_upload.to_csv('man_upload.csv')